In [32]:
import pandas as pd

# Set these according to the option you used:
# Example if you uploaded files via files.upload():
true_path = "/content/True.csv"
fake_path = "/content/Fake.csv"  # Corrected path: changed 'fake.csv' to 'Fake.csv'

# If you mounted drive, set true_path/fake_path to your drive paths instead.

true_df = pd.read_csv(true_path, engine='python', on_bad_lines='skip')
fake_df = pd.read_csv(fake_path)

print("=== TRUE ===")
print("Shape:", true_df.shape)
print("Columns:", true_df.columns.tolist())
display(true_df.head(3))

print("\n=== FAKE ===")
print("Shape:", fake_df.shape)
print("Columns:", fake_df.columns.tolist())
display(fake_df.head(3))

=== TRUE ===
Shape: (21417, 4)
Columns: ['title', 'text', 'subject', 'date']


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"



=== FAKE ===
Shape: (23481, 4)
Columns: ['title', 'text', 'subject', 'date']


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"


#Data Cleaning & Label Preparation

In [34]:
import pandas as pd

# Load files (adjust if needed)
true_df = pd.read_csv("/content/True.csv", engine='python', on_bad_lines='skip')
fake_df = pd.read_csv("/content/Fake.csv", engine='python', on_bad_lines='skip')

# 1. Add labels
true_df["label"] = 1
fake_df["label"] = 0

# 2. Combine
df = pd.concat([true_df, fake_df], ignore_index=True)

# 3. Basic cleaning
def clean_text(text):
    text = str(text)
    text = text.replace("\n", " ").replace("\r", " ")
    text = " ".join(text.split())   # remove extra spaces
    return text

df["text"] = df["text"].apply(clean_text)
df["title"] = df["title"].apply(clean_text)

# 4. Shuffle
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print("FINAL DATA SHAPE:", df.shape)
df.tail(5)

FINAL DATA SHAPE: (44898, 5)


,title,text,subject,date,label
44893,Nigeria says U.S. agrees delayed $593 million ...,ABUJA (Reuters) - The United States has formal...,worldnews,"December 27, 2017",1
44894,Boiler Room #62 – Fatal Illusions,Tune in to the Alternate Current Radio Network...,Middle-east,"June 29, 2016",0
44895,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...,I m convinced the Freedom From Religion group ...,Government News,"Feb 27, 2016",0
44896,Republican tax plan would deal financial hit t...,WASHINGTON (Reuters) - The Republican tax plan...,politicsNews,"November 2, 2017",1
44897,U.N. refugee commissioner says Australia must ...,SYDNEY (Reuters) - The U.N. High Commissioner ...,worldnews,"November 1, 2017",1


#Train a baseline ML classifier (TF-IDF + Logistic Regression)

 1. Split dataset into train & test

In [35]:
from sklearn.model_selection import train_test_split

X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", len(X_train))
print("Test size :", len(X_test))

Train size: 35918
Test size : 8980


2. TF-IDF Vectorizer

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english", max_features=50000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)

3. Train Logistic Regression Model

In [37]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=300)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=300)


4. Evaluate the model


In [38]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test_tfidf)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))



Accuracy: 0.987305122494432

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      4696
           1       0.98      0.99      0.99      4284

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


Confusion Matrix:
 [[4621   75]
 [  39 4245]]


In [39]:
# -------------------------------
# 1. Split dataset into train & test
# -------------------------------
from sklearn.model_selection import train_test_split

X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", len(X_train))
print("Test size :", len(X_test))

# -------------------------------
# 2. TF-IDF Vectorizer
# -------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english", max_features=50000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)

# -------------------------------
# 3. Train Logistic Regression Model
# -------------------------------
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=300)
model.fit(X_train_tfidf, y_train)

# -------------------------------
# 4. Evaluate the model
# -------------------------------
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test_tfidf)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Train size: 35918
Test size : 8980

Accuracy: 0.987305122494432

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      4696
           1       0.98      0.99      0.99      4284

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


Confusion Matrix:
 [[4621   75]
 [  39 4245]]


Because your dataset is highly imbalanced (many more fake examples), the classifier learned to prioritize the majority class. Before we jump to Transformers, we should:

Inspect misclassified examples (to understand why the model fails).

Create a reliable validation split (so hyperparameters + early stopping are trustworthy).

Decide how to handle imbalance (resampling, class weights, focal loss, or mix).

Prepare the Hugging Face dataset + tokenizer for fine-tuning RoBERTa.

#Inspect misclassifications (why model fails)

1.Inspect misclassification




In [40]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import numpy as np

# if your combined df is named df (from earlier)
# And you have X_test, y_test, y_pred from previous step
test_df = pd.DataFrame({"text": X_test.values, "label": y_test.values, "pred": y_pred})
fn = test_df[(test_df.label==1) & (test_df.pred==0)]   # real but predicted fake
fp = test_df[(test_df.label==0) & (test_df.pred==1)]   # fake but predicted real

print("False negatives (real->fake):", len(fn))
print("False positives (fake->real):", len(fp))

# show examples (title not in X_test by default, so we pull titles from df if needed)
# If you kept 'title' in df, create test_df with titles. Otherwise just text.
display(fn.head(20)[["text"]])
display(fp.head(20)[["text"]])


False negatives (real->fake): 39
False positives (fake->real): 75


,text
8,(Reuters) - North Korean leader Kim Jong Un ha...
638,WASHINGTON (Reuters) - U.S. President Donald T...
850,WASHINGTON (Reuters) - The budget proposal unv...
869,(Reuters) - Here are some of the highlights of...
1081,HAVANA (Reuters) - U.S. President Barack Obama...
1357,LOS ANGELES (Reuters) - The Emmy awards show w...
1621,MELBOURNE (Reuters) - Andy Murray thinks he pl...
1849,(Reuters) - Former U.S. president George W. Bu...
1891,WASHINGTON (Reuters) - National security advis...
2537,"KARACHI, Pakistan (Reuters) - Muslims in Pakis..."


,text
81,As progressivism gets more of a stronghold in ...
388,Dr. Ben Carson was interviewed by a local TV s...
392,If you haven t seethe viral video of a takedow...
503,These 4 hostages must certainly feel a sense o...
540,How very progressive While Hillary insults wom...
623,Thierry Meyssan Voltaire NetIt is a scandal wi...
740,Just when you thought the 2014 election result...
977,Here is the list of Republicans who won t supp...
1040,This entire situation with Puerto Rico reminds...
1246,McConnell was shut down completely in this deb...


2.Create Train, Validation, and Test Splits

In [41]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.25, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print("TRAIN:", train_df.shape)
print("VAL:", val_df.shape)
print("TEST:", test_df.shape)

train_df.to_csv("/content/train.csv", index=False)
val_df.to_csv("/content/val.csv", index=False)
test_df.to_csv("/content/test.csv", index=False)
print("Saved /content/train.csv, /content/val.csv, /content/test.csv")



TRAIN: (33673, 5)
VAL: (5612, 5)
TEST: (5613, 5)
Saved /content/train.csv, /content/val.csv, /content/test.csv


3.Fix Class Imbalance (IMPORTANT)

In [42]:
from sklearn.utils import resample

# Separate classes
real = train_df[train_df.label == 1]
fake = train_df[train_df.label == 0]

# Oversample real
real_upsampled = resample(real, replace=True, n_samples=len(fake), random_state=42)

# Combine
train_df_balanced = pd.concat([fake, real_upsampled]).sample(frac=1, random_state=42)

print("Balanced training size:", train_df_balanced.shape)
print("Fake count:", sum(train_df_balanced.label == 0))
print("Real count:", sum(train_df_balanced.label == 1))


Balanced training size: (35220, 5)
Fake count: 17610
Real count: 17610


#Fine-Tune Transformer Model (RoBERTa-base)

1.Prepare HuggingFace Dataset + Tokenization

In [43]:
!pip install -q transformers datasets accelerate evaluate sentencepiece

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

# Use RoBERTa tokenizer
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create HF datasets
train_hf = Dataset.from_pandas(train_df_balanced.reset_index(drop=True))
val_hf   = Dataset.from_pandas(val_df.reset_index(drop=True))
test_hf  = Dataset.from_pandas(test_df.reset_index(drop=True))

dataset = DatasetDict({
    "train": train_hf,
    "validation": val_hf,
    "test": test_hf
})

# Tokenization function
def tokenize(batch):
    return tokenizer(
        [t + " . " + x for t, x in zip(batch["title"], batch["text"])],
        padding="max_length",
        truncation=True,
        max_length=256   # choose 256 to reduce GPU usage
    )

dataset = dataset.map(tokenize, batched=True)

# Keep only model-required columns
dataset = dataset.remove_columns(['title', 'text', 'subject', 'date'])
dataset = dataset.rename_column("label", "labels")
dataset.set_format(type="torch")

dataset

Map:   0%|          | 0/35220 [00:00<?, ? examples/s]

Map:   0%|          | 0/5612 [00:00<?, ? examples/s]

Map:   0%|          | 0/5613 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 35220
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5612
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5613
    })
})

#Fine-Tune RoBERTa-base (with class weights)

In [22]:
import os
os.environ["WANDB_DISABLED"] = "true"   # disable wandb

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from datasets import DatasetDict
import torch.nn as nn

model_name = "roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# --- CLASS WEIGHTS ---
# Get labels from the prepared dataset directly
labels_array = np.array(dataset["train"]["labels"])
class_weights_cpu = compute_class_weight("balanced", classes=np.unique(labels_array), y=labels_array)

# --- TRAINER WITH WEIGHTED LOSS ---
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Move class_weights to the same device as the model
        class_weights = torch.tensor(class_weights_cpu, dtype=torch.float).to(model.device)

        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# --- TRAINING ARGUMENTS ---
training_args = TrainingArguments(
    output_dir="./roberta-fake-news",
    eval_strategy="epoch", # Changed evaluation_strategy to eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,           # Increase to 4–5 if GPU strong
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    fp16=True                     # Mixed precision for speed (if GPU supports)
)

# --- METRICS ---
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

# --- TRAIN ---
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1860653381.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000000,0.000012,1.000000,1.000000,1.000000,1.000000
2,0.000000,0.000005,1.000000,1.000000,1.000000,1.000000
3,0.000000,0.000004,1.000000,1.000000,1.000000,1.000000


TrainOutput(global_step=3042, training_loss=0.005068878904737781, metrics={'train_runtime': 486.306, 'train_samples_per_second': 50.043, 'train_steps_per_second': 6.255, 'total_flos': 3201535321620480.0, 'train_loss': 0.005068878904737781, 'epoch': 3.0})

#Test Evaluation on the Test Set

In [23]:
metrics = trainer.evaluate(dataset["test"])
print(metrics)

preds = trainer.predict(dataset["test"])
y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))


{'eval_loss': 4.107681888854131e-06, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 4.0617, 'eval_samples_per_second': 268.36, 'eval_steps_per_second': 16.988, 'epoch': 3.0}

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       677
           1       1.00      1.00      1.00       413

    accuracy                           1.00      1090
   macro avg       1.00      1.00      1.00      1090
weighted avg       1.00      1.00      1.00      1090


Confusion Matrix:
 [[677   0]
 [  0 413]]


#Build FAISS Vector Store for RAG

In [24]:
!pip install faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 76.6 MB/s eta 0:00:00


prepare embeddings

In [25]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# use ONLY real news articles for evidence
real_news = df[df['label']==1]['text'].tolist()

embeddings = embed_model.encode(real_news, convert_to_numpy=True)
embeddings = embeddings.astype('float32')

print("Embedding shape:", embeddings.shape)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding shape: (3308, 384)


Build FAISS index

In [26]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("FAISS index built with:", index.ntotal, "documents")


FAISS index built with: 3308 documents


RAG Retrieval Function

In [27]:
def retrieve_evidence(query, k=3):
    q_emb = embed_model.encode([query], convert_to_numpy=True).astype('float32')
    distances, indices = index.search(q_emb, k)
    return [real_news[i] for i in indices[0]]


Complete Fake News Explanation Function

In [30]:
from transformers import AutoTokenizer

clf_tokenizer = tokenizer   # your RoBERTa tokenizer
clf_model = trainer.model   # your fine-tuned model

def predict_and_explain(news_text):
    # 1. Classification
    inputs = clf_tokenizer(news_text, return_tensors='pt', truncation=True, max_length=256)

    # Move inputs to the same device as the model
    inputs = {k: v.to(clf_model.device) for k, v in inputs.items()}

    outputs = clf_model(**inputs)
    pred = outputs.logits.argmax().item()
    label = "FAKE" if pred == 0 else "REAL"

    # 2. Evidence Retrieval
    evidence = retrieve_evidence(news_text, k=3)

    # 3. Print results
    print("Prediction:", label)
    print("\nEvidence from similar REAL news articles:")
    for i, ev in enumerate(evidence):
        print(f"\n--- Evidence {i+1} ---\n")
        print(ev[:500], "...")

In [31]:
test_news = """
President signs a new executive order on gun control...
"""

predict_and_explain(test_news)


Prediction: FAKE

Evidence from similar REAL news articles:

--- Evidence 1 ---

WASHINGTON (Reuters) - U.S. President Donald Trump on Monday issued an executive order revoking limits imposed by predecessor Barack Obama on the transfer of surplus military equipment to local law enforcement agencies, the White House said. Obama had curtailed the equipment transfer program after law enforcement officers using military-style armored vehicles and guns confronted protesters in Ferguson, Missouri, in 2014 following the fatal police shooting of a black teenager. Trump’s executive o ...

--- Evidence 2 ---

NEW YORK (Reuters) - U.S. President Donald Trump said on Thursday he had signed an executive order that would allow the United States to ramp up sanctions on North Korean firms in an effort to dissuade Pyongyang from pursuing its nuclear missile program. “Our new executive order will cut off sources of revenue that fund North Korea’s efforts to develop the deadliest weapons known to humanki